<a href="https://colab.research.google.com/github/jmershov/SureStartFutureMakers/blob/main/Day_23_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
dataset = tf.keras.utils.get_file('ac1imdb_v1', url, untar=True, cache_dir='.', cache_subdir='')
print(dataset)

./ac1imdb_v1


In [3]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
print(dataset_dir)

./aclImdb


In [4]:
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'train', 'test', 'README']

In [5]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['unsupBow.feat',
 'urls_pos.txt',
 'neg',
 'labeledBow.feat',
 'urls_unsup.txt',
 'urls_neg.txt',
 'pos',
 'unsup']

In [6]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [7]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [8]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',  batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)
print(len(raw_train_ds))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
625


In [11]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(32):
    print("Review ", text_batch.numpy()[i])
    print("Label ", label_batch.numpy()[i])

Review  b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />There\'s a

In [12]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',  batch_size=batch_size, validation_split=0.2, subset='validation', seed=seed)
print(len(raw_val_ds))

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
157


In [13]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/test',  batch_size=batch_size)
print(len(raw_test_ds))

Found 25000 files belonging to 2 classes.
782


In [19]:
def custom_standardization(input_data):
  lower = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lower, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')


In [20]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [21]:
train_text = raw_train_ds.map(lambda x, y:x)
vectorize_layer.adapt(train_text)

In [23]:
def vectorize_text(text, label):
  # Add an innermost dimension for batch
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [24]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b"The idea ia a very short film with a lot of information. Interesting, entertaining and leaves the viewer wanting more. The producer has produced a short film of excellent quality that cannot be compared to any other short film that I have seen. I have rated this film at the highest possible rating. I also recommend that it is shown to office managers and business people in any establishment. What comes out of it is the fact that people with ideas are never listened to, their voice is never heard. It is a lesson to be learned by any office that wants to go forward. I hope that the produced will produce a second part to this 'idea'. I look forward to viewing the sequence. Once again congrats to Halaqah media in producing a film of excellence and quality with a lesson in mind.", shape=(), dtype=string)
Label pos
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[   2,  314,    1,    4,   52,  350,   19,   16,    4,  171,    5,
        1574,  217,

In [25]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  silent
 313 --->  night
Vocabulary size: 10000


In [26]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [27]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [28]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim), 
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [30]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 13s 15ms/step - loss: 0.6611 - binary_accuracy: 0.6896 - val_loss: 0.6107 - val_binary_accuracy: 0.7778
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.5453 - binary_accuracy: 0.8033 - val_loss: 0.4952 - val_binary_accuracy: 0.8234
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.4420 - binary_accuracy: 0.8460 - val_loss: 0.4179 - val_binary_accuracy: 0.8480
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3765 - binary_accuracy: 0.8672 - val_loss: 0.3719 - val_binary_accuracy: 0.8622
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3330 - binary_accuracy: 0.8801 - val_loss: 0.3434 - val_binary_accuracy: 0.8678
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3038 - binary_accuracy: 0.8905 - val_loss: 0.3245 - val_binary_accuracy: 0.8716
Epoch 7/10
625/625 [==============================] - 3s 6ms/s

In [31]:
# Evaluate on test set
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 7s 9ms/step - loss: 0.3101 - binary_accuracy: 0.8730
Loss:  0.3101290762424469
Accuracy:  0.8730000257492065
